In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl

from biopandas.pdb import PandasPdb

In [379]:
# User input, global variables, the part to be calculated
chain = 'A'    # which chain
residue_start  = 1   # which residue starts
residue_end    = 439    # which residue ends


In [449]:
# path
# 1st pdb, the important one, the base 
path1 = '/media/tcheng/4900fa07-fb34-4b94-93d3-0d932f25d1c2/26S/Moises/full_data/cryosparc/J76_conform1.1/rmsd2/full_docked5-close-coot-5.pdb'

# 2nd pdb, the one to be compared to
path2 = '/media/tcheng/4900fa07-fb34-4b94-93d3-0d932f25d1c2/26S/Moises/full_data/cryosparc/J76_conform1.1/rmsd2/7w3j_aligned_to_G-M.pdb'

# out_path
path3 = ' ' 

In [450]:
# reading in the first pdb into python dataframe, and the file to be modified
p1 = PandasPdb().read_pdb(path1)

# the second pdb, the pdb to be compared
p2 = PandasPdb().read_pdb(path2)

In [451]:
# some formating of the first pdb, select main chain only
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]

# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

#p2_mainchain

In [363]:
# find the residues that intersected 
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
intersect

array([ 23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
        49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
        62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,
        75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,
        88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
       140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 20

In [314]:
# calculate per residue rmsd
size = np.size(np.arange(residue_start, residue_end + 1, 1))
rmsd1 = np.zeros((size,2))
j=0
for i in intersect:
#for i in np.arange(residue_start, residue_end + 1, 1):
    z = PandasPdb.rmsd(p1_mainchain.loc[(p1.df['ATOM']['residue_number'] == i)],p2_mainchain.loc[(p2.df['ATOM']['residue_number'] == i)], s='main chain')
    rmsd1[j]= i, round(z,2) 
    j = j + 1
    #print(i)
#PandasPdb.rmsd(p1_chain.loc[(p1.df['ATOM']['residue_number'] == 59)],p2_chain.loc[(p2.df['ATOM']['residue_number'] == 59)], s='main chain')
rmsd1_df = pd.DataFrame(rmsd1, columns=['residue', 'rmsd'])      # convert it to a dataframe
rmsd1_df['rmsd']

0      10.28
1      10.75
2      11.00
3      10.97
4      10.60
       ...  
434     0.00
435     0.00
436     0.00
437     0.00
438     0.00
Name: rmsd, Length: 439, dtype: float64

In [315]:
# weird problem with using len() function
k1 = p1.df['ATOM']['chain_id'].shape
k2 = rmsd1_df['rmsd'].shape

# set b-fac to zero first
#p1.df['ATOM']['b_factor']  = 0

# the real replacement
for i in np.arange(k1[0]):
    if (p1.df['ATOM']['chain_id'][i] == chain):
        p1.df['ATOM']['b_factor']  = 0.0
        p1.df['ATOM']['b_factor'].astype(float) 

for i in np.arange(k1[0]):
    for j in np.arange(k2[0]):
        #print(i)
        #print(j)
        if (p1.df['ATOM']['chain_id'][i] == chain) & (p1.df['ATOM']['residue_number'][i] == rmsd1_df['residue'][j]):
            p1.df['ATOM']['b_factor'][i] = rmsd1_df['rmsd'][j]  
            #print(p1.df['ATOM']['b_factor'][i])
p1.df['ATOM']['b_factor'].dtype

/home/tcheng/miniconda3/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


dtype('float64')

In [316]:
# just to check
p1.df['ATOM']['b_factor']

0       10.28
1       10.28
2       10.28
3       10.28
4       10.28
        ...  
2781     7.85
2782     7.85
2783     7.85
2784     7.85
2785     7.85
Name: b_factor, Length: 2786, dtype: float64

In [317]:
p1.to_pdb('/media/tcheng/4900fa07-fb34-4b94-93d3-0d932f25d1c2/26S/Moises/full_data/cryosparc/J76_conform1.1/rmsd3/rmsd_new_F.pdb')      # saving to a pdb

In [ ]:
#### Below is for whole chain RMSD #### 

In [452]:
# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]

# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

#p2_mainchain

In [453]:
# A cell for each chain
#chain A first
chain = 'A'    # which chain
residue_start  = 157   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainA = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainA = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainA, p2_mainchainA, s='main chain')
print('RMSD chain A: %.4f Angstrom' % rmsd)

RMSD chain A: 6.0221 Angstrom


In [454]:
# A cell for each chain
#chain B 
chain = 'B'    # which chain
residue_start  = 167   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainB = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainB = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainB, p2_mainchainB, s='main chain')
print('RMSD chain B: %.4f Angstrom' % rmsd)

RMSD chain B: 6.2455 Angstrom


In [455]:
# A cell for each chain
#chain C 
chain = 'C'    # which chain
residue_start  = 137   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainC = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainC = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainC, p2_mainchainC, s='main chain')
print('RMSD chain C: %.4f Angstrom' % rmsd)

RMSD chain C: 6.0195 Angstrom


In [456]:
# A cell for each chain
#chain D 
chain = 'D'    # which chain
residue_start  = 154   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainD = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainD = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainD, p2_mainchainD, s='main chain')
print('RMSD chain D: %.4f Angstrom' % rmsd)

RMSD chain D: 5.7362 Angstrom


In [457]:
# A cell for each chain
#chain E 
chain = 'E'    # which chain
residue_start  = 121   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainE = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainE = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainE, p2_mainchainE, s='main chain')
print('RMSD chain E: %.4f Angstrom' % rmsd)

RMSD chain E: 19.1138 Angstrom


In [458]:
# A cell for each chain
#chain F 
chain = 'F'    # which chain
residue_start  = 168   # which residue starts
residue_end    = 500    # which residue ends

# some formating of the first pdb, select main chain only, sometimes only the mainchains atoms exist
p1_chain = p1.df['ATOM'].loc[(p1.df['ATOM']['chain_id'] == chain)] 
p1_mainchain = p1_chain.loc[(p1.df['ATOM']['atom_name'] == 'C')|(p1.df['ATOM']['atom_name'] == 'O')|(p1.df['ATOM']['atom_name'] == 'N')|(p1.df['ATOM']['atom_name'] == 'CA')]
# some formating of the second pdb. 
p2_chain = p2.df['ATOM'].loc[(p2.df['ATOM']['chain_id'] == chain)]
p2_mainchain = p2_chain.loc[(p2.df['ATOM']['atom_name'] == 'C')|(p2.df['ATOM']['atom_name'] == 'O')|(p2.df['ATOM']['atom_name'] == 'N')|(p2.df['ATOM']['atom_name'] == 'CA')]

# find the intersects
size = np.size(np.arange(residue_start, residue_end + 1, 1))
full_list = pd.DataFrame(np.arange(residue_start, residue_end + 1, 1), columns=['residue_number'])
    
p2_residue_df = pd.DataFrame(p2_chain['residue_number'], columns=['residue_number'])
p2_unique_df = p2_residue_df.drop_duplicates(subset='residue_number')

p1_residue_df = pd.DataFrame(p1_chain['residue_number'], columns=['residue_number'])
p1_unique_df = p1_residue_df.drop_duplicates(subset='residue_number')

intersect = np.intersect1d(p1_unique_df['residue_number'], p2_unique_df['residue_number'])
#np.setdiff1d(full_list['residue_number'], p2_unique_df['residue_number'])
#intersect

# select the intersecting residues
p1_mainchainF = p1_mainchain[p1_mainchain['residue_number'].isin(intersect)]
p2_mainchainF = p2_mainchain[p2_mainchain['residue_number'].isin(intersect)]

rmsd = PandasPdb.rmsd(p1_mainchainF, p2_mainchainF, s='main chain')
print('RMSD chain F: %.4f Angstrom' % rmsd)

RMSD chain F: 9.4152 Angstrom


In [459]:
#combine all df
p1_combined = pd.concat([p1_mainchainA, p1_mainchainB, p1_mainchainC, p1_mainchainD, p1_mainchainE, p1_mainchainF])
p2_combined = pd.concat([p2_mainchainA, p2_mainchainB, p2_mainchainC, p2_mainchainD, p2_mainchainE, p2_mainchainF])

# overal rmsd
rmsd = PandasPdb.rmsd(p1_combined, p2_combined, s='main chain')
print('RMSD combined %.4f Angstrom' % rmsd)

#p1_combined

RMSD combined 9.8479 Angstrom
